In [1]:
!pip install ddgs
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.4 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [2]:
from ddgs import DDGS
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import re
import warnings
import numpy as np
warnings.simplefilter("ignore", category=FutureWarning)

In [3]:
def scan_prices(prompt, val, item):
  ddgs = DDGS()
  results = list(ddgs.text(query=prompt, results = val))
  info = []
  urls = []
  for i in results:
    text = " ".join(f"{key} {value}" for key, value in i.items())
    currency_symbols = r'[$€£¥₣]'
    extracted_text = re.findall(fr'[^;\n]*?{currency_symbols}\d[\d,.]*[^;\n]*', text)
    for j in extracted_text:
      if (j.strip()):
        j = j.lower()
        info.append(j)
        urls.append(i.get("href", ""))
  return info, urls

In [4]:
def load_model():
  bert_model_name = "deepset/roberta-base-squad2"
  bert_model = AutoModelForQuestionAnswering.from_pretrained(bert_model_name)
  bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
  bert_qa = pipeline("question-answering", model = bert_model, tokenizer = bert_tokenizer)
  return bert_qa

In [5]:
exchange_rates = {}
exchange_rates['$'] = 1
currencies = ['€','£','¥','₣']
for currency in currencies:
  query = f"conversion of {currency} to USD"
  ddgs = DDGS()
  results = list(ddgs.text(query=query, results = 5))
  for i in results:
    for key, value in i.items():
      if isinstance(value, str):
        match = re.search(fr'1\s+{currency.upper()}\s*=\s*([\d.]+)\s*USD', value)
        if match:
          exchange_rate = float(match.group(1))
          exchange_rates[currency] = exchange_rate
print(exchange_rates)

{'$': 1}


In [6]:
def isfloat(val):
  try:
    float(val)
    return True
  except Exception as e:
    return False

In [10]:
exchange_rates = {}
exchange_rates['$'] = 1
currencies = ['€','£','¥','₣']
bert_qa = load_model()
ddgs = DDGS()
for currency in currencies:
  prompt = f"conversion of {currency} to USD"
  results = list(ddgs.text(query=prompt, results = 5))
  prompt = "what is the numerical exchange rate of CHF to USD"
  answers = []
  answer_scores = []
  for result in results:
    # Convert results to string not dictionary
    text = " ".join(f"{key} {value}, " for key, value in result.items() if (currency in str(value)))
    if (not text.strip()):
      continue
    answer = bert_qa(prompt, text)
    if (isfloat(answer['answer']) == True):
      answers.append(answer['answer'])
      answer_scores.append(answer['score'])
      print(f"answer for {currency} to USD is: {answer['answer']}")
      print(f"score is: {answer['score']}")
      exchange_rates[currency] = float(answer['answer'])
# print("answers is: ", answers)
# print("answer_scores are: ", answer_scores)
# print("conversion rate confidence score is: ", max(answer_scores))
# print("conversion rate is: ", answers[np.argmax(answer_scores)])

Device set to use cpu


answer for £ to USD is: 52.06
score is: 2.5755733531696023e-05


In [11]:
def extract_prices_with_symbols(text):
  matches = re.findall(r'([$€£¥₣])\s?\d[\d,]*\.?\d*', text)
  return matches

In [12]:
while True:
  item = input("What object's price do you want to look for?")
  prompt = f"What is the unit price of {item}?"
  bert_qa = load_model()
  x = True
  while (x):
    val = input("How expansive of a search do you want?")
    if (val.isdigit() == True):
      val = int(val)
      x = False
    else:
      val = input("Please enter a valid search number")
  info, urls = scan_prices(prompt, val, item)
  answers = []
  url_answers = []
  for count, i in enumerate(info):
    context = i
    answer = bert_qa(prompt, context)
    if (answer['score'] > 0.70):
      answers.append(answer["answer"])
      print(f"answer is: {answer['answer']}")
      print(f"url is: {urls[count]}")
      print(f"score is: {answer['score']}")
      url_answers.append(urls[count])
  # prices = []
  # for ans in answers:
  #   matches = extract_prices_with_symbols(ans)
  #   for match in matches:
  #     amount = exchange_rates[match[0]]
  #     price = float(match[0]) * amount
  #     prices.append(price)
  if (len(answers) > 0):
    print("answers are: ", answers)
    # print(f"price range is: {min(prices)}-{max(prices)}")
    print(f"urls are {urls}")
  else:
    print("Could not find any reliable answers")

What object's price do you want to look for?pens


Device set to use cpu


How expansive of a search do you want?10
Could not find any reliable answers
What object's price do you want to look for?pencils


Device set to use cpu


How expansive of a search do you want?10
answer is: $ 0.28 per pencil
url is: https://brainly.com/question/2044452
score is: 0.7671781778335571
answer is: $0.27
url is: https://brainly.com/question/3208137
score is: 0.8358449339866638
answers are:  ['$ 0.28 per pencil', '$0.27']
urls are ['https://brainly.com/question/2044452', 'https://www.gauthmath.com/solution/1813032697269413/If-6-pencils-cost-1-80-what-is-the-unit-price-of-the-pencils-The-unit-price-is-s', 'https://brainly.com/question/3208137', 'https://www.reddit.com/r/pencils/comments/10pbuod/cost_per_pencil_what_do_you_consider_to_be_a_deal/', 'https://www.wyzant.com/resources/answers/259483/it_costs_3_24_for_a_pack_of_12_pencils_find_the_unit_price_in_dollars_per_pencil_if_necessary_round_your_answer_to_the_nearest_cent', 'https://www.gauthmath.com/solution/1812002346497030/A-pack-of-20-pencils-5-costs-4-80-What-is-the-unit-price-per-pencil-', 'https://www.quora.com/9-pens-and-5-pencils-cost-3-2-and-7-pens-and-8-pencils-costs

KeyboardInterrupt: Interrupted by user